In [1]:
!pip install session_info
!pip install pycaret

In [13]:
pip install pycaret[tuners]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 14.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 34.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 26.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 KB 15.0 MB/s eta 0:00:00
  Created wheel for future: filename=future-0

#### Import librairies

In [1]:
import pandas as pd
import os
import re


from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset , random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

import random

from pycaret.classification import *
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix

import session_info

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
session_info.show()

#### Set the random seed for reproducibility

In [2]:
seed = 64
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#### Get the data

In [3]:
raw_data= pd.read_csv('data/train.csv', low_memory=False)
data_test= pd.read_csv('data/test.csv', low_memory=False)

In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Columns: 365 entries, Patient_ID to Type_of_Venom_Allergy_IGE_Venom
dtypes: float64(322), int64(32), object(11)
memory usage: 8.3+ MB


#### Looking which are the targets to predict

In [5]:
missing_cols = set(raw_data.columns) ^ set(data_test.columns)
print(missing_cols)
len(missing_cols)

{'Respiratory_Allergy', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram', 'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Food_Allergy_Tree_Nuts', 'Type_of_Respiratory_Allergy_ARIA', 'Venom_Allergy', 'Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Respiratory_Allergy_IGE_Pollen_Tree', 'Type_of_Food_Allergy_Mammalian_Milk', 'Type_of_Food_Allergy_TPO', 'Type_of_Food_Allergy_Other', 'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'trustii_id', 'Type_of_Food_Allergy_Fish', 'Type_of_Food_Allergy_Aromatics', 'Type_of_Food_Allergy_Other_Legumes', 'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach', 'Type_of_Respiratory_Allergy_IGE_Molds_Yeast', 'Type_of_Food_Allergy_Peanut', 'Type_of_Respiratory_Allergy_GINA', 'Type_of_Food_Allergy_Oral_Syndrom', 'Type_of_Food_Allergy_Shellfish', 'Food_Allergy', 'Type_of_Food_Allergy_Egg', 'Severe_Allerg

30

## Data Pre-processing

### Preprocessing for the train set

In [6]:
liste_of_Targets =['Allergy_Present', 'Severe_Allergy', 'Respiratory_Allergy', 'Food_Allergy', 'Venom_Allergy',
                     'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_CONJ', 
                     'Type_of_Respiratory_Allergy_GINA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                     'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Type_of_Respiratory_Allergy_IGE_Pollen_Tree',
                     'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach',
                     'Type_of_Respiratory_Allergy_IGE_Molds_Yeast', 'Type_of_Food_Allergy_Aromatics', 'Type_of_Food_Allergy_Other',
                     'Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish',
                     'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Food_Allergy_Mammalian_Milk', 
                     'Type_of_Food_Allergy_Oral_Syndrom', 'Type_of_Food_Allergy_Other_Legumes', 'Type_of_Food_Allergy_Peanut',
                     'Type_of_Food_Allergy_Shellfish', 'Type_of_Food_Allergy_TPO', 'Type_of_Food_Allergy_Tree_Nuts',
                     'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Venom_Allergy_IGE_Venom']
def preprocessing_data(df):
    df = df.drop('Food_Type_0', axis =1)
    df.replace(-1, 0, inplace=True)
    data_noNAN = df.fillna(-1)
    # obtain Targets
    Targets = data_noNAN.loc[:,liste_of_Targets]
    # filter feautures
    X1=data_noNAN.loc[:, ['Chip_Type','Age','Gender','French_Residence_Department','Blood_Month_sample']]
    X= data_noNAN.iloc[:, 8:-29]
    data = pd.concat( [X1, X] , axis=1)
    # handle the 'Treatment_of_rhinitis' feature
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].astype(str)
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].str.replace('.0', '', regex=True)
    
    ##  Get_dummies of the 'object' type columns
    
    columns_to_encode = ['Chip_Type', 'French_Residence_Department', 'French_Region',
         'Treatment_of_athsma', 'Age_of_onsets',
       'General_cofactors', 'Treatment_of_atopic_dematitis','Treatment_of_rhinitis']
    
    ### Split the columns using multiple delimiters and create dummy columns
    dummy_dfs = []
    for col in columns_to_encode:
        # Split the data in the column that use  delimiters
        data[col] = data[col].astype(str)
        data[col] = data[col].apply(lambda x: [i.strip() for i in re.split('[,.]', x)])

        # Create dummy columns
        dummy_df = pd.get_dummies(data[col].apply(pd.Series).stack(), prefix=f"{col}", prefix_sep='_').groupby(level=0).sum()
        dummy_dfs.append(dummy_df)

    ### Concatenate the original DataFrame with the dummy columns
    df_final = pd.concat([data] + dummy_dfs, axis=1)

    ### Drop the original columns from the final dataset
    df_final.drop(columns=columns_to_encode, inplace=True)
    
    # Converting all values into 'float16' type
    encode_data = df_final.astype('float16')
    print(encode_data.info())
    
    return encode_data,Targets

In [7]:
encode_data,Targets = preprocessing_data(raw_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Columns: 467 entries, Age to Treatment_of_rhinitis_9
dtypes: float16(467)
memory usage: 2.7 MB
None


### Preprocessing for the test set

In [8]:
def preprocessing_data_test(df):
    df = df.drop('Food_Type_0', axis =1)
    df.replace(-1, 0, inplace=True)
    data_test_noNAN = df.fillna(-1)
    # filter feautures
    X1=data_test_noNAN.loc[:, ['Chip_Type']]
    X= data_test_noNAN.iloc[:, 5:]
    data = pd.concat( [X1, X] , axis=1)
    # handle the 'Treatment_of_rhinitis' feature
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].astype(str)
    data['Treatment_of_rhinitis'] = data['Treatment_of_rhinitis'].str.replace('.0', '', regex=True)
    # handle 'Age_of_onsets' which don't have the same format in data test and train
    data['Age_of_onsets'] = data['Age_of_onsets'].astype(str)

    
    ##  Get_dummies of the 'object' type columns
    
    columns_to_encode = ['Chip_Type', 'French_Residence_Department', 'French_Region',
         'Treatment_of_athsma', 'Age_of_onsets',
       'General_cofactors', 'Treatment_of_atopic_dematitis','Treatment_of_rhinitis']
    
    ### Split the columns using multiple delimiters and create dummy columns
    dummy_dfs = []
    for col in columns_to_encode:
        # Split the data in the column that use  delimiters
        data[col] = data[col].astype(str)
        data[col] = data[col].apply(lambda x: [i.strip() for i in re.split('[,.]', x)])

        # Create dummy columns
        dummy_df = pd.get_dummies(data[col].apply(pd.Series).stack(), prefix=f"{col}", prefix_sep='_').groupby(level=0).sum()
        dummy_dfs.append(dummy_df)

    ### Concatenate the original DataFrame with the dummy columns
    df_final = pd.concat([data] + dummy_dfs, axis=1)

    ### Drop the original columns from the final dataset
    df_final.drop(columns=columns_to_encode, inplace=True)
    
    # Converting all values into 'float16' type
    encode_data = df_final.astype('float16')
    print(encode_data.info())
    
    return encode_data


In [9]:
encode_data_test = preprocessing_data_test(data_test)
missing_cols = set(encode_data.columns) ^ set(encode_data_test.columns)
print(missing_cols)
len(missing_cols)
encode_data_test = encode_data_test.reindex(columns=encode_data.columns, fill_value=0).astype('float16')
encode_data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Columns: 444 entries, Age to Treatment_of_rhinitis_9
dtypes: float16(444)
memory usage: 508.3 KB
None
{'French_Residence_Department_deptNNN', 'French_Residence_Department_deptQQQ', 'French_Residence_Department_deptOOO', 'French_Residence_Department_deptMMM', 'French_Residence_Department_deptHHH', 'General_cofactors_11', 'French_Residence_Department_deptIII', 'Treatment_of_athsma_8', 'French_Residence_Department_deptDD', 'French_Region_regionN', 'French_Residence_Department_deptPPP', 'French_Region_regionO', 'French_Residence_Department_deptTTT', 'French_Residence_Department_deptAAAA', 'French_Residence_Department_deptUU', 'French_Residence_Department_deptDDD', 'French_Residence_Department_deptK', 'French_Residence_Department_deptU', 'French_Residence_Department_deptW', 'French_Residence_Department_deptRRR', 'French_Residence_Department_deptCCCC', 'Treatment_of_atopic_dematitis_7', 'Treatment_of_athsma_10', 'French_

#### Pycaret part

In [18]:
import os

def obtain_stack_methods_imbl(encode_data, Y):
    
    liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']
        
    folder_path = f'Test_Imbl_stacked_multi_tuned_selection'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    for column in Y.columns:
        if column not in liste_column_monovalue :
            rows_with_9 = Y[column].isin([9])
            Targets_without_9 = Y[column][~rows_with_9].reset_index(drop=True)
            X = encode_data[~rows_with_9].reset_index(drop=True)

            dataset= pd.concat([X,Targets_without_9], axis = 1)

            setup_model_1 = setup(data= dataset, target= column, train_size=0.68, fix_imbalance=True, fix_imbalance_method='SMOTE')
            lightgbm= create_model('lightgbm')
            tuned_lightgbm, tuner= tune_model(lightgbm, n_iter = 70, optimize = 'F1', fold = 8, search_library = 'tune-sklearn', search_algorithm = 'hyperopt', return_tuner=True)
            
            file_name = f"{column}_imblearn_stacked_multi_tuned_selection"
            file_path = os.path.join(folder_path, file_name)
            save_model(tuned_lightgbm, file_path)
        else:
            continue

In [19]:
obtain_stack_methods_imbl(encode_data, Targets)

(_Trainable pid=66047) [LightGBM] [Warning] bagging_fraction is set=0.49247705785158014, subsample=1.0 will be ignored. Current value: bagging_fraction=0.49247705785158014 [repeated 2x across cluster]
(_Trainable pid=66047) [LightGBM] [Warning] feature_fraction is set=0.532383789354755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.532383789354755 [repeated 2x across cluster]
(_Trainable pid=66047) [LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3 [repeated 2x across cluster]
(_Trainable pid=66047) [LightGBM] [Warning] bagging_fraction is set=0.49247705785158014, subsample=1.0 will be ignored. Current value: bagging_fraction=0.49247705785158014 [repeated 2x across cluster]
(_Trainable pid=66047) [LightGBM] [Warning] feature_fraction is set=0.532383789354755, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.532383789354755 [repeated 2x across cluster]
(_Trainable pid=66047) [LightG

2023-06-26 20:44:39,911	INFO tune.py:1111 -- Total run time: 1440.25 seconds (1440.19 seconds for the tuning loop).


Transformation Pipeline and Model Successfully Saved


### Generate predictions from classifiers and include them into the dataset + evaluation 

In [20]:
data= pd.read_csv('data/train.csv', low_memory=False)
data_test= pd.read_csv('data/test.csv', low_memory=False)

In [23]:
List_methods_imbl=['SMOTE']

def obtain_pred(data, folder_path):
    liste=[]
    for subdir, dirs, files in os.walk(folder_path):
        for file in files:
            print("Calling model:", file) 
            model = load_model(os.path.join(subdir, os.path.splitext(file)[0]))
            values=predict_model(model, data= data.reset_index())
            file_name = os.path.splitext(file)[0]
            values_name = '{}_{}'.format(file_name, 'values')
            globals()[values_name] = values
            globals()[values_name] = globals()[values_name].rename(columns={'prediction_label': 'pred_label ' + str(values_name), 'prediction_score': 'pred_score ' + str(values_name) })
            liste.append(globals()[values_name])
        return liste
    
def obtain_all_pred_methods(data, list_of_methods):
    all_lists = {} 
    for method in list_of_methods:
        folder_path = f'Test_Imbl_{method}_tuned_stacked_and_calibrated'
        all_lists[f"{method}_tuned"] = obtain_pred(data, folder_path)
    return all_lists
    

In [21]:
def obtain_pred(data, folder_path):
    liste=[]
    for subdir, dirs, files in os.walk(folder_path):
        for file in files:
            print("Calling model:", file) 
            model = load_model(os.path.join(subdir, os.path.splitext(file)[0]))
            values=predict_model(model, data= data.reset_index())
            file_name = os.path.splitext(file)[0]
            values_name = '{}_{}'.format(file_name, 'values')
            globals()[values_name] = values
            globals()[values_name] = globals()[values_name].rename(columns={'prediction_label': 'pred_label ' + str(values_name), 'prediction_score': 'pred_score ' + str(values_name) })
            liste.append(globals()[values_name])
        return liste

In [22]:
dico_df= obtain_pred(encode_data, folder_path='Test_Imbl_stacked_multi_tuned_selection')

Calling model: Type_of_Respiratory_Allergy_ARIA_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Food_Allergy_Fish_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Severe_Allergy_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Venom_Allergy_IGE_Venom_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Venom_Allergy_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


...

Calling model: Type_of_Venom_Allergy_ATCD_Venom_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Food_Allergy_Tree_Nuts_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_IGE_Dander_Animals_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Food_Allergy_TPO_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Respiratory_Allergy_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


In [23]:
print(len(dico_df))
dico_df[0]

#len(dico_df.values())

27


,index,Age,Gender,Blood_Month_sample,Rural_or_urban_area,Sensitization,Skin_Symptoms,Act_d_1,Act_d_2,Act_d_5,...,Treatment_of_atopic_dematitis_7,Treatment_of_atopic_dematitis_9,Treatment_of_rhinitis_0,Treatment_of_rhinitis_1,Treatment_of_rhinitis_2,Treatment_of_rhinitis_3,Treatment_of_rhinitis_4,Treatment_of_rhinitis_9,pred_label Type_of_Respiratory_Allergy_ARIA_imblearn_stacked_multi_tuned_selection_values,pred_score Type_of_Respiratory_Allergy_ARIA_imblearn_stacked_multi_tuned_selection_values
0,0,15.0,0.0,7.0,1.0,1.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.5113
1,1,72.0,1.0,5.0,9.0,1.0,9.0,0.000000,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.5013
2,2,67.0,1.0,6.0,9.0,0.0,9.0,0.000000,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.5012
3,3,13.0,1.0,9.0,1.0,1.0,9.0,0.000000,2.150391,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0.5113
4,4,28.0,1.0,12.0,9.0,1.0,1.0,0.000000,74.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.5009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2984,2984,21.0,1.0,3.0,9.0,1.0,9.0,0.000000,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.5014
2985,2985,3.0,0.0,1.0,9.0,0.0,9.0,0.000000,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0.5051
2986,2986,15.0,0.0,12.0,1.0,1.0,1.0,0.000000,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0.5072
2987,2987,11.0,1.0,4.0,0.0,1.0,0.0,0.770020,0.099976,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.5113


In [28]:

liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']

merged_df = pd.DataFrame()
for i,df in enumerate(dico_df):
    if i==0:
        merged_df = df
    else:
        last_two_columns = df.iloc[:, -2:]
        merged_df = pd.concat([merged_df, last_two_columns], axis=1)

added_list = ['pred_label ' + s + '_imblearn_stacked_multi_tuned_selection_values' for s in liste_column_monovalue]
zeros_data = pd.DataFrame(0, index=np.arange(len(merged_df)), columns= added_list)
merged_df = pd.concat([merged_df, zeros_data], axis=1)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2989 entries, 0 to 2988
Columns: 524 entries, index to pred_label Type_of_Food_Allergy_Cereals_&_Seeds_imblearn_stacked_multi_tuned_selection_values
dtypes: float32(467), float64(27), int32(1), int64(29)
memory usage: 6.6 MB


In [25]:
print(len(list_of_merged_df))
print(list_of_key)

NameError: name 'list_of_merged_df' is not defined

In [26]:
data_true= pd.read_csv('data/train.csv', low_memory=False)

In [29]:
merged_df.head()

,index,Age,Gender,Blood_Month_sample,Rural_or_urban_area,Sensitization,Skin_Symptoms,Act_d_1,Act_d_2,Act_d_5,...,pred_label Type_of_Food_Allergy_Tree_Nuts_imblearn_stacked_multi_tuned_selection_values,pred_score Type_of_Food_Allergy_Tree_Nuts_imblearn_stacked_multi_tuned_selection_values,pred_label Type_of_Respiratory_Allergy_IGE_Dander_Animals_imblearn_stacked_multi_tuned_selection_values,pred_score Type_of_Respiratory_Allergy_IGE_Dander_Animals_imblearn_stacked_multi_tuned_selection_values,pred_label Type_of_Food_Allergy_TPO_imblearn_stacked_multi_tuned_selection_values,pred_score Type_of_Food_Allergy_TPO_imblearn_stacked_multi_tuned_selection_values,pred_label Respiratory_Allergy_imblearn_stacked_multi_tuned_selection_values,pred_score Respiratory_Allergy_imblearn_stacked_multi_tuned_selection_values,pred_label Type_of_Food_Allergy_Other_imblearn_stacked_multi_tuned_selection_values,pred_label Type_of_Food_Allergy_Cereals_&_Seeds_imblearn_stacked_multi_tuned_selection_values
0,0,15.0,0.0,7.0,1.0,1.0,0.0,0.0,0.000000,0.0,...,0,0.8970,0,0.9795,0,0.5001,1,0.9927,0,0
1,1,72.0,1.0,5.0,9.0,1.0,9.0,0.0,0.000000,0.0,...,0,0.9359,0,0.9838,0,0.5001,1,0.9311,0,0
2,2,67.0,1.0,6.0,9.0,0.0,9.0,0.0,0.000000,0.0,...,0,0.9353,0,0.9963,0,0.5001,1,0.7721,0,0
3,3,13.0,1.0,9.0,1.0,1.0,9.0,0.0,2.150391,0.0,...,0,0.7661,1,0.9875,0,0.5000,1,0.9982,0,0
4,4,28.0,1.0,12.0,9.0,1.0,1.0,0.0,74.000000,0.0,...,0,0.8013,1,0.9860,0,0.5001,1,0.9813,0,0


#### Evaluation binary for each methods

In [30]:
import numpy as np
target_columns = ['Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Food_Allergy_Other_Legumes', 'Allergy_Present', 'Type_of_Food_Allergy_Mammalian_Milk', 'Type_of_Food_Allergy_Other',
                  'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach', 'Venom_Allergy', 'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Food_Allergy', 'Type_of_Food_Allergy_Oral_Syndrom','Type_of_Food_Allergy_Tree_Nuts', 'Severe_Allergy',
                  'Type_of_Food_Allergy_Aromatics', 'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Peanut',
                  'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish', 'Type_of_Respiratory_Allergy_GINA', 'Respiratory_Allergy', 'Type_of_Food_Allergy_TPO',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Tree', 'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Respiratory_Allergy_IGE_Molds_Yeast',
                  'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Food_Allergy_Shellfish']


f1_scores=[]
below_threshold_columns=[]
threshold =0.85
for column in target_columns:
    rows_with_9 = data_true[column].isin([9])
    Targets_without_9 = data_true[column][~rows_with_9]
    merged_df_removal = merged_df[~rows_with_9]

    merged_df_mod = pd.concat([merged_df_removal, Targets_without_9] , axis=1).reset_index(drop=True)
    y_true = merged_df_mod[column]
    y_pred = merged_df_mod['pred_label '+ column + '_imblearn_stacked_multi_tuned_selection_values']

    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Calculate recall
    #recall = recall_score(y_true, y_pred)

    # Calculate F1 score
    if (data_true[column] == 1).any():
        f1 = f1_score(y_true, y_pred,average='macro')
    else:
        f1 = f1_score(y_true, y_pred, pos_label=0)
    cm = confusion_matrix(y_true, y_pred)

    # Display the metrics and confusion matrix
    print(f"Metrics for {column}:")
    print(f"Accuracy: {accuracy}")
    #print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print("Confusion Matrix:")
    print(cm)
    print()
    if f1 != 0:
        f1_scores.append(f1) # Append the F1 score to the list
    if f1 < threshold:
        below_threshold_columns.append(column)

mean_f1 = sum(f1_scores) / len(f1_scores)  # Calculate the mean of F1 scores

print(f"Mean F1 Score: {mean_f1}")
print(below_threshold_columns)
print('')

Metrics for Type_of_Food_Allergy_Cereals_&_Seeds:
Accuracy: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1607]]

Metrics for Type_of_Food_Allergy_Other_Legumes:
Accuracy: 0.9651524579962664
F1 Score: 0.7453537068477646
Confusion Matrix:
[[1522   35]
 [  21   29]]

Metrics for Allergy_Present:
Accuracy: 0.9647527003979534
F1 Score: 0.9486509443201423
Confusion Matrix:
[[ 356   35]
 [  27 1341]]

Metrics for Type_of_Food_Allergy_Mammalian_Milk:
Accuracy: 0.9838207840696951
F1 Score: 0.7792115831748044
Confusion Matrix:
[[1564   18]
 [   8   17]]

Metrics for Type_of_Food_Allergy_Other:
Accuracy: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1607]]

Metrics for Type_of_Respiratory_Allergy_IGE_Mite_Cockroach:
Accuracy: 0.9807564698075647
F1 Score: 0.9805425885485188
Confusion Matrix:
[[818   9]
 [ 20 660]]

Metrics for Venom_Allergy:
Accuracy: 0.9899631983941117
F1 Score: 0.7947566468907932
Confusion Matrix:
[[2937   19]
 [  11   22]]

Metrics for Type_of_Respiratory_Allergy_ARIA:
Accuracy: 0.90179163901

## Generate prediction for the test set

In [31]:
original_validation_data = pd.read_csv('data/test.csv')

data_test = original_validation_data.set_index('trustii_id')

In [32]:
encode_data_test = preprocessing_data_test(data_test)
missing_cols = set(encode_data.columns) ^ set(encode_data_test.columns)
print(missing_cols)
len(missing_cols)
encode_data_test = encode_data_test.reindex(columns=encode_data.columns, fill_value=0).astype('float16')
encode_data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 586 entries, 1 to 1282
Columns: 443 entries, Gender to Treatment_of_rhinitis_9
dtypes: float16(443)
memory usage: 527.8 KB
None
{'French_Residence_Department_deptNNN', 'French_Residence_Department_deptQQQ', 'French_Residence_Department_deptOOO', 'French_Residence_Department_deptMMM', 'French_Residence_Department_deptHHH', 'General_cofactors_11', 'French_Residence_Department_deptIII', 'Age', 'Treatment_of_athsma_8', 'French_Residence_Department_deptDD', 'French_Region_regionN', 'French_Residence_Department_deptPPP', 'French_Region_regionO', 'French_Residence_Department_deptTTT', 'French_Residence_Department_deptAAAA', 'French_Residence_Department_deptUU', 'French_Residence_Department_deptDDD', 'French_Residence_Department_deptK', 'French_Residence_Department_deptU', 'French_Residence_Department_deptW', 'French_Residence_Department_deptRRR', 'French_Residence_Department_deptCCCC', 'Treatment_of_atopic_dematitis_7', 'Treatment_of_athsma_10

In [33]:
dico_df= obtain_pred(encode_data_test.reset_index(), folder_path='Test_Imbl_stacked_multi_tuned_selection')

Calling model: Type_of_Respiratory_Allergy_ARIA_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Food_Allergy_Fish_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Severe_Allergy_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Venom_Allergy_IGE_Venom_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Venom_Allergy_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


...

Calling model: Type_of_Venom_Allergy_ATCD_Venom_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Food_Allergy_Tree_Nuts_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Respiratory_Allergy_IGE_Dander_Animals_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Type_of_Food_Allergy_TPO_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


Calling model: Respiratory_Allergy_imblearn_stacked_multi_tuned_selection.pkl
Transformation Pipeline and Model Successfully Loaded


In [34]:
print(len(dico_df))


27


In [35]:
liste_column_monovalue=['Type_of_Food_Allergy_Other','Type_of_Food_Allergy_Cereals_&_Seeds']

merged_df = pd.DataFrame()
for i,df in enumerate(dico_df):
    if i==0:
        merged_df = df
    else:
        last_two_columns = df.iloc[:, -2:]
        merged_df = pd.concat([merged_df, last_two_columns], axis=1)

added_list = ['pred_label ' + s + '_imblearn_stacked_multi_tuned_selection_values' for s in liste_column_monovalue]
zeros_data = pd.DataFrame(0, index=np.arange(len(merged_df)), columns= added_list)
merged_df = pd.concat([merged_df, zeros_data], axis=1)
merged_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 586 entries, 0 to 585
Columns: 525 entries, index to pred_label Type_of_Food_Allergy_Cereals_&_Seeds_imblearn_stacked_multi_tuned_selection_values
dtypes: float32(467), float64(27), int32(2), int64(29)
memory usage: 1.3 MB


In [36]:
merged_df.head()

,index,trustii_id,Age,Gender,Blood_Month_sample,Rural_or_urban_area,Sensitization,Skin_Symptoms,Act_d_1,Act_d_2,...,pred_label Type_of_Food_Allergy_Tree_Nuts_imblearn_stacked_multi_tuned_selection_values,pred_score Type_of_Food_Allergy_Tree_Nuts_imblearn_stacked_multi_tuned_selection_values,pred_label Type_of_Respiratory_Allergy_IGE_Dander_Animals_imblearn_stacked_multi_tuned_selection_values,pred_score Type_of_Respiratory_Allergy_IGE_Dander_Animals_imblearn_stacked_multi_tuned_selection_values,pred_label Type_of_Food_Allergy_TPO_imblearn_stacked_multi_tuned_selection_values,pred_score Type_of_Food_Allergy_TPO_imblearn_stacked_multi_tuned_selection_values,pred_label Respiratory_Allergy_imblearn_stacked_multi_tuned_selection_values,pred_score Respiratory_Allergy_imblearn_stacked_multi_tuned_selection_values,pred_label Type_of_Food_Allergy_Other_imblearn_stacked_multi_tuned_selection_values,pred_label Type_of_Food_Allergy_Cereals_&_Seeds_imblearn_stacked_multi_tuned_selection_values
0,0,1,0.0,1.0,6.0,9.0,1.0,0.0,0.0,0.000000,...,0,0.7275,1,0.9536,0,0.5001,1,0.9989,0,0
1,1,4,0.0,1.0,7.0,9.0,1.0,1.0,0.0,0.109985,...,0,0.5736,1,0.9803,0,0.5000,1,0.9876,0,0
2,2,5,0.0,0.0,10.0,1.0,1.0,1.0,0.0,0.000000,...,0,0.8599,1,0.9657,0,0.5001,1,0.9863,0,0
3,3,7,0.0,1.0,8.0,0.0,1.0,0.0,0.0,0.000000,...,0,0.8625,0,0.9827,0,0.5001,0,0.7082,0,0
4,4,8,0.0,0.0,4.0,1.0,0.0,9.0,0.0,0.000000,...,0,0.7847,0,0.9969,0,0.5001,0,0.8765,0,0


In [37]:
target_columns = ['Type_of_Food_Allergy_Cereals_&_Seeds', 'Type_of_Food_Allergy_Other_Legumes', 'Allergy_Present', 'Type_of_Food_Allergy_Mammalian_Milk', 'Type_of_Food_Allergy_Other',
                  'Type_of_Respiratory_Allergy_IGE_Mite_Cockroach', 'Venom_Allergy', 'Type_of_Respiratory_Allergy_ARIA', 'Type_of_Respiratory_Allergy_IGE_Pollen_Gram',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Herb', 'Food_Allergy', 'Type_of_Food_Allergy_Oral_Syndrom','Type_of_Food_Allergy_Tree_Nuts', 'Severe_Allergy',
                  'Type_of_Food_Allergy_Aromatics', 'Type_of_Venom_Allergy_IGE_Venom', 'Type_of_Venom_Allergy_ATCD_Venom', 'Type_of_Respiratory_Allergy_CONJ', 'Type_of_Food_Allergy_Peanut',
                  'Type_of_Food_Allergy_Egg', 'Type_of_Food_Allergy_Fish', 'Type_of_Respiratory_Allergy_GINA', 'Respiratory_Allergy', 'Type_of_Food_Allergy_TPO',
                  'Type_of_Respiratory_Allergy_IGE_Pollen_Tree', 'Type_of_Food_Allergy_Fruits_and_Vegetables', 'Type_of_Respiratory_Allergy_IGE_Molds_Yeast',
                  'Type_of_Respiratory_Allergy_IGE_Dander_Animals', 'Type_of_Food_Allergy_Shellfish']

original_validation_data = pd.read_csv('data/test.csv')
for elem in target_columns:
    original_validation_data[elem]=  merged_df['pred_label '+ elem + '_imblearn_stacked_multi_tuned_selection_values']

In [38]:
len(original_validation_data)

586

In [39]:
original_validation_data.head(30)

,trustii_id,Patient_ID,Chip_Code,Chip_Type,Chip_Image_Name,Age,Gender,Blood_Month_sample,French_Residence_Department,French_Region,...,Type_of_Food_Allergy_Egg,Type_of_Food_Allergy_Fish,Type_of_Respiratory_Allergy_GINA,Respiratory_Allergy,Type_of_Food_Allergy_TPO,Type_of_Respiratory_Allergy_IGE_Pollen_Tree,Type_of_Food_Allergy_Fruits_and_Vegetables,Type_of_Respiratory_Allergy_IGE_Molds_Yeast,Type_of_Respiratory_Allergy_IGE_Dander_Animals,Type_of_Food_Allergy_Shellfish
0,1,PMP0156,22 262C 3858,ISAC_V2,NaN,8.0,1.0,6.0,deptBBB,regionJ,...,0,0,1,1,0,1,0,1,1,0
1,4,PCR0234,02AHX0DC,ALEX,02AHX0DC.bmp,14.0,1.0,7.0,deptL,regionD,...,0,0,1,1,0,0,0,0,1,1
2,5,PCR0532,02AUN372,ALEX,02AUN372.png,32.0,0.0,10.0,deptUUU,regionF,...,0,0,1,1,0,1,0,1,1,0
3,7,GJH0147,EKF3830_4,ISAC_V2,EKF3830_4_2200444337_2023_2_17_11_58_24.bmp,65.0,1.0,8.0,deptQ,regionF,...,0,0,0,0,0,0,0,0,0,0
4,8,TXV0009,881204001164,ISAC_V1,1G20027_2_881204001164_2012_4_25_18_32_58.bmp,5.0,0.0,4.0,deptII,regionC,...,0,0,0,0,0,0,0,0,0,0
5,9,PCR0118,02AFA752,ALEX,NaN,49.0,0.0,1.0,deptXXX,regionI,...,0,0,0,1,0,0,0,0,0,0
6,10,QVW0214,AB02627_3,ISAC_V1,NaN,6.0,1.0,2.0,deptY,regionD,...,0,0,1,1,0,1,0,1,1,0
7,15,TXV0157,881602013302,ISAC_V1,BAF4027_4_881602013302_2016_2_23_16_38_11.bmp,13.0,1.0,2.0,deptRR,regionB,...,0,0,1,1,0,0,0,0,0,0
8,18,WQW0190,223112546,ISAC_V2,END0E30_1_223112546_2023_1_3_16_20_19.bmp,12.0,0.0,11.0,deptOO,regionL,...,0,0,0,0,0,0,0,0,0,0
9,23,TXV0282,881903001372,ISAC_V1,CXG1527_3_881903001372_2019_3_14_3_51_59.bmp,8.0,0.0,3.0,deptEE,regionC,...,0,0,1,1,0,0,0,0,0,0


In [40]:
original_validation_data.to_csv(f"Submission_ML_pycaret_multi_tuned_selection_binary.csv", index=False, encoding='UTF-8')

#### Logics is the same than the first submission but an fix_imbalanced_method is set to give models more samples from the minority class and less from the majority class.
#### one notebook for the the best sampling methods where different models obtain with different methods and setup are stacked.